This example notebook demonstrates how to write VAME results to an NWB file, using the [ndx-vame](https://github.com/catalystneuro/ndx-vame) extension.

It will also include:
- [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) metadata
- [PoseEstimation](https://github.com/rly/ndx-pose) data

Let's start by importing the necessary modules:

In [ ]:
import datetime
import numpy as np
import json
from pynwb import NWBFile, NWBHDF5IO
from pynwb.file import Subject
from ndx_pose import PoseEstimationSeries, PoseEstimation
from ndx_vame import VAMEProject, LatentSpaceSeries, MotifSeries, CommunitySeries

Next, we create the NWB file object containing basic metadata and pose estimation data:

In [ ]:
# initialize an NWBFile object
nwbfile = NWBFile(
    session_description="session_description",
    identifier="identifier",
    session_start_time=datetime.datetime.now(datetime.timezone.utc),
)

# add a subject to the NWB file
subject = Subject(subject_id="subject1", species="Mus musculus")
nwbfile.subject = subject

# create a device for the camera
camera1 = nwbfile.create_device(
    name="camera1",
    description="camera for recording behavior",
    manufacturer="manufacturer",
)

# a PoseEstimationSeries represents the estimated position of a single marker.
# in this example, we have three PoseEstimationSeries: one for the body and one for each front paw.
data = np.random.rand(100, 2)  # num_frames x (x, y) but can be (x, y, z)
confidence = np.random.rand(100)  # a confidence value for every frame
reference_frame = "(0,0,0) corresponds to ..."
front_left_paw = PoseEstimationSeries(
    name="front_left_paw",
    description="Marker placed around fingers of front left paw.",
    data=data,
    unit="pixels",
    rate=10.,
    reference_frame=reference_frame,
    confidence=confidence,
)

data = np.random.rand(100, 2)  # num_frames x (x, y) but can be (x, y, z)
confidence = np.random.rand(100)  # a confidence value for every frame
body = PoseEstimationSeries(
    name="body",
    description="Marker placed on center of body.",
    data=data,
    unit="pixels",
    rate=10.,
    reference_frame=reference_frame,
    confidence=confidence,
)

data = np.random.rand(100, 2)  # num_frames x (x, y) but can be num_frames x (x, y, z)
confidence = np.random.rand(100)  # a confidence value for every frame
front_right_paw = PoseEstimationSeries(
    name="front_right_paw",
    description="Marker placed around fingers of front right paw.",
    data=data,
    unit="pixels",
    rate=10.,
    reference_frame=reference_frame,
    confidence=confidence,
)

# create a PoseEstimation object
pose_estimation = PoseEstimation(
    name="PoseEstimation",
    pose_estimation_series=[front_left_paw, body, front_right_paw],
    description="Estimated positions of front paws of subject1 using DeepLabCut.",
    original_videos=["path/to/camera1.mp4"],
    dimensions=np.array([[640, 480]], dtype="uint16"),  # pixel dimensions of the video
    devices=[camera1],
    scorer="DLC_resnet50_openfieldOct30shuffle1_1600",
    source_software="DeepLabCut",
    source_software_version="2.3.8",
)

# create a "behavior" processing module to store the PoseEstimation and Skeletons objects
behavior_pm = nwbfile.create_processing_module(
    name="behavior",
    description="processed behavioral data",
)
behavior_pm.add(pose_estimation)

Now we can include VAME-related data to the NWB file:

- Latent space data
- Motifs data
- Communities data

In [ ]:
# VAME data
latent_data = np.random.rand(100, 30)  # (n_samples, z_dims)
data_motifs = np.random.randint(0, 15, size=100)  # (n_samples)
data_communities = np.random.randint(0, 3, size=100)  # (n_samples)

latent_space_series = LatentSpaceSeries(
    name="LatentSpaceSeries",
    data=latent_data,
    rate=10.,
)

motif_series = MotifSeries(
    name="MotifSeries",
    data=data_motifs,
    rate=10.,
    algorithm="hmm",
    latent_space_series=latent_space_series,
)
community_series = CommunitySeries(
    name="CommunitySeries",
    data=data_communities,
    rate=10.,
    algorithm="hierarchical_clustering",
    motif_series=motif_series,
)

config_dict = {}  # Configuration dictionary from VAME project
vame_project = VAMEProject(
    name="VAMEProject",
    pose_estimation=pose_estimation,
    latent_space_series=latent_space_series,
    motif_series=motif_series,
    community_series=community_series,
    vame_config=json.dumps(config_dict),
)

behavior_pm.add(vame_project)

In [ ]:
# write the NWBFile to disk
path = "test_vame.nwb"
with NWBHDF5IO(path, mode="w") as io:
    io.write(nwbfile)

# read the NWBFile from disk
io = NWBHDF5IO(path, mode="r")
nwbfile = io.read()
print(nwbfile.processing["behavior"]["VAMEProject"])